# Base blueprint subregions

After working on the LMVJV indicators, I noticed that some of the subregions don't quite line up with the LMVJV subregions. I would prefer to have the flexibility to modify the subregions based on code in case for testing so I started making code for subregions in my spare time. This will be mostly coded, with limited hand editing steps at the end, which will make it easier to modify if partners request modifications. 

Created by Amy Keister, last run by Amy Keister on 6 December 2021.

In [44]:
import os
import arcpy

In [45]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace= 
OutWorkspace = r"E:\WORKING\BaseBlueprint\2022\1_SubregionsAndExtent\SESubregions2022.gdb"

In [116]:
# define final outputs
ExtentOut = r"E:\WORKING\BaseBlueprint\2022\1_SubregionsAndExtent\BaseBlueprintExtent2022g.tif"
ShapeOut = r"E:\WORKING\BaseBlueprint\2022\1_SubregionsAndExtent"

In [47]:
# define inputs
Eco= r"F:\GIS_DATA\Boundaries\Ecoregions\Omernik\us_eco_l4_state_boundaries\us_eco_l4.shp"
H4= r"F:\GIS_DATA\WaterResources\NHD\WBD_National_GDB\WBD_National_GDB.gdb\WBDHU4"
MaR= r"F:\GIS_DATA\Boundaries\Ecoregions\Marine\MarineEcoregions_L3_Shapefile\NA_Marine_Ecoregions_L3\data\Marine_Ecoregions_updated\Marine_Ecoregions_updated.shp"
State= r"F:\GIS_DATA\Boundaries\States\tl_2021_us_state\tl_2021_us_state.shp"
SA = r"F:\GIS_DATA\SALCC_BLUEPRINT\Version_2021\Blueprint_2021_Data_Download_20220204\1_ExtentLayers\SouthAtlantic2021Extent.shp"
Square= r"E:\WORKING\BaseBlueprint\2022\1_SubregionsAndExtent\SquareForTesting.gdb\square"
snap= r"F:\GIS_DATA\LanduseLandcover\NLCD\NLCD_Land_Cover_L48_20190424\NLCD_2016_Land_Cover_L48_20190424_SECASbuff_N83.img"

## Start Analysis

In [48]:
# Set the workspace where I want the output to go
arcpy.env.workspace = OutWorkspace

In [49]:
print(arcpy.env.workspace)

E:\WORKING\BaseBlueprint\2022\1_SubregionsAndExtent\SESubregions2022.gdb


## Make copys of ecoregions

In [50]:
# make a copy of the ecoregions so I can add a field
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.management.CopyFeatures(Eco, "eco", '', None, None, None)

## Do some initial grouping of ecoregions

In [51]:
# THE OBJECTID WILL CHANGE WITH FRESH DOWNLOADS OF THIS DATA! NEED TO RECHECK THIS
# I'm using the eco OBJECT ID insead of doing hand editing to to address what I'm calling "islands" where
# the ecoregions have smaller non-contiguous polygons

codeblock = """
def Reclass(OID):
    if OID in [266]:
        return "Edwards Plateau in High Plains and Tablelands"
    elif OID in [2285,2284]:
        return "MAV in Texas-Louisiana Coast"
    elif OID in [1317,1291]:
        return "WGCP in MAV"
    elif OID in [2816]:
        return "River Hills in MAV"
    elif OID in [1925,1924]:
        return "S App in Interior Plateau"
    elif OID in [1827]:
        return "N Piedmont in S App"
    elif OID in [1812,1811]:
        return "Blue Ridge in S Piedmont"
    elif OID in [1814,1815,6895,6896,6897,6908]:
        return "Blue Ridge in N Piedmont"
    elif OID in [2827]:
        return "MAV in Interior Plateau"
    elif OID in [1813]:
        return "Blue Ridge in combo N/S Piedmont"
    elif OID in [1401]:
        return "N Piedmont in C Atlantic Coastal Plain"
"""

In [52]:
# add field to show "islands" of ecoregions that are not contiguous with the main ecoregion
arcpy.management.CalculateField("eco", "Islands", "Reclass(!OBJECTID!)", "PYTHON3", codeblock, "TEXT")                            

<Result 'eco'>

In [53]:
# set codeblock for next step
codeblock = """
def Reclass(sn):
    if str(sn) in ('Texas','Oklahoma','Missouri','Arkansas','Louisiana','Mississippi','Alabama','Georgia','Tennessee','Kentucky','Florida','South Carolina','North Carolina','Virginia','West Virginia'):
        return "SECAS"
"""

In [54]:
# select SECAS states
arcpy.management.CalculateField("eco", "SECAS", "Reclass(!STATE_NAME!)", "PYTHON3", codeblock, "TEXT")                            

<Result 'eco'>

In [55]:
# set codeblock for next step
codeblock = """
def Reclass(L3,L4):
    if str(L3) in ('Chihuahuan Deserts','Arizona/New Mexico Mountains'):
        return "Chihuahuan Deserts"
    elif str(L3) in ('High Plains','Southwestern Tablelands'):
        return "High Plains and Tablelands"
    elif str(L3) in ('Central Great Plains','Cross Timbers','Flint Hills'):
        return "Plains and Timbers"
    elif str(L3) == 'Southern Texas Plains':
        return "South Texas Plains"
    elif str(L3) in ('Edwards Plateau'):
        return "Edwards Plateau"
    elif str(L3) in ('Texas Blackland Prairies','East Central Texas Plains'):
        return "Texas Blackland Prairies"
    elif str(L3) == 'Western Gulf Coastal Plain':
        return "Gulf Coastal Prairies"
    elif str(L3) in ('South Central Plains'):
        return "West Gulf Coastal Plain"
    elif str(L3) == 'Mississippi Alluvial Plain':
        return "Mississippi Alluvial Valley"
    elif str(L3) in ('Boston Mountains','Arkansas Valley','Ouachita Mountains'):
        return "Ouachita"
    elif str(L3) == 'Ozark Highlands':
        return "Ozarks and Plains"
    elif str(L4) in ('Osage Cuestas','Cherokee Plains'):
        return "Ozarks and Plains"
    elif str(L4) in ('Loess Flats and Till Plains','Claypan Prairie','Wooded Osage Plains','Upper Mississippi Alluvial Plain'):
        return "North Missouri"
    elif str(L3) == 'Western Corn Belt Plains':
        return "North Missouri"
    elif str(L3) in ('Interior Plateau'):
        return "Interior Plateau"
    elif str(L4) in ('Green River-Southern Wabash Lowlands','Caseyville Hills'):
        return "Interior Plateau"     
    elif str(L3) in ('Southeastern Plains','Southern Coastal Plain','Middle Atlantic Coastal Plain','Southern Florida Coastal Plain'):
        return "SE Plains"
    elif str(L4) in ('Loess Plains','Southern Rolling Plains','Baton Rouge Terrace'):
        return "SE Plains"
    elif str(L3) in ('Piedmont','Northern Piedmont'):
        return "Piedmont"
    elif str(L3) in ('Ridge and Valley','Southwestern Appalachians','Central Appalachians','Western Allegheny Plateau','Blue Ridge'):
        return "Appalachians"     
    elif str(L4) in ('River Hills'):
        return "River Hills"    
    elif str(L4) in ('Bluff Hills'):
        return "Bluff Hills"    
    elif str(L4) in ('Middle Mississippi Alluvial Plain'):
        return "Ozarks and Plains"
    elif str(L4) in ('Wabash-Ohio Bottomlands'):
        return "Wabash-Ohio Bottomlands"
"""

In [56]:
# do some initial grouping of ecoregions, based on code above
arcpy.management.CalculateField("eco", "EcoCombine", "Reclass(!US_L3NAME!,!US_L4NAME!)", "PYTHON3", codeblock, "TEXT")                            

<Result 'eco'>

## Make copys of HUC4

In [57]:
# make a copy of the HUC4 so I can add a field
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.management.CopyFeatures(H4, "H4", '', None, None, None)

## Do some initial grouping of HUC4

In [58]:
# set codeblock for next step
codeblock = """
def Reclass(h4):
    if str(h4) in ('0204','0208','0301','0302','0303','0304','0305','0306','0307','0308','0309','0310','0311','0312','0313','0314','0316','0317','0318','0808','0809','1201','1204','1203','1209','1210','1211','1309','2101','2102','0809'):
        return "SECAS"
"""

In [59]:
# select near shore HUC4s that are a part of the SECAS area
arcpy.management.CalculateField("H4", "SECASh", "Reclass(!HUC4!)", "PYTHON3", codeblock, "TEXT")   

<Result 'H4'>

## Union

In [60]:
# union eco and H4 layers that I did prelimiary grouping on above
arcpy.analysis.Union(["eco", "H4"], "ecoH4", "ALL", None, "NO_GAPS")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\1_SubregionsAndExtent\\SESubregions2022.gdb\\ecoH4'>

In [61]:
# narrow down to closer to SECAS area based on the secas states and secas near shore HUC4s
arcpy.analysis.Select("ecoH4", "ecoH4a", "STATE_NAME NOT IN ('Maryland', 'New Jersey', 'Delaware', 'New York', 'Pennsylvania') And (SECASh = 'SECAS' Or SECAS = 'SECAS')")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\1_SubregionsAndExtent\\SESubregions2022.gdb\\ecoH4a'>

In [62]:
# set codeblock for next step
codeblock = """
def Reclass(e,h,f):
    if str(e) in ('Piedmont') and str(h) in ('Edisto-Santee','Altamaha-St. Marys','Ogeechee-Savannah','Apalachicola','Alabama') and f != -1:
        return "South Piedmont"
    elif str(e) in ('Piedmont') and str(h) in ('Chowan-Roanoke','Neuse-Pamlico','Potomac','Delaware-Mid Atlantic Coastal','Cape Fear','Pee Dee','Lower Chesapeake') and f != -1:
        return "North Piedmont"
    elif str(e) in ('Bluff Hills') and str(h) in ('Lower Mississippi-St. Francis')and f != -1:
        return "Mississippi Alluvial Valley"
    elif str(e) in ('River Hills') and str(h) in ('Upper Mississippi-Salt') and f != -1:
        return "North Missouri"
    elif str(e) in ('River Hills') and f != -1:
        return "Ozarks and Plains"
    elif str(e) in ('River Hills') and str(h) in ('Upper Mississippi-Salt') and f != -1:
        return "North Missouri"
    elif str(e) in ('Wabash-Ohio Bottomlands') and str(h) in ('Green') and f != -1:
        return "Interior Plateau"
    elif str(e) in ('Wabash-Ohio Bottomlands') and str(h) in ('Lower Mississippi-Hatchie') and f != -1:
        return "Mississippi Alluvial Valley"
    elif str(e) in ('SE Plains') and str(h) in ('Lower Chesapeake','Chowan-Roanoke','Neuse-Pamlico','Potomac','Delaware-Mid Atlantic Coastal'):
        return "North Atlantic Coastal Plain"       
    elif str(e) in ('SE Plains') and str(h) in ('Cape Fear','Pee Dee'):
        return "Central Atlantic Coastal Plain"
    elif str(e) in ('SE Plains') and str(h) in ('Edisto-Santee','Altamaha-St. Marys','Ogeechee-Savannah'):
        return "South Atlantic Coastal Plain"
    elif str(e) in ('SE Plains') and str(h) in ('Apalachicola','Ochlockonee','Suwannee'):
        return "East Gulf Coastal Plain Sub" 
    elif str(e) in ('SE Plains') and str(h) in ('St. Johns','Peace-Tampa Bay','Southern Florida'):
        return "Florida Peninsula" 
    elif str(e) in ('Wabash-Ohio Bottomlands') and f != -1:
        return "Interior Plateau"
    elif str(h) in ('Puerto Rico and Virgin Islands') and f == -1:
        return "Puerto Rico and Virgin Islands"
    elif str(e) in ('Appalachians') and str(h) in ('Middle Ohio','Upper Ohio','Kentucky-Licking','Monongahela','Kanawha','Big Sandy-Guyandotte','Chowan-Roanoke','Potomac','Lower Chesapeake','Scioto','Pee Dee') and f != -1:
        return "North Appalachians"
    elif str(e) in ('Appalachians') and str(h) in ('Alabama','Middle Tennessee-Elk','Apalachicola','Mobile-Tombigbee','Cumberland','Middle Tennessee-Hiwassee','Upper Tennessee','Ogeechee-Savannah','Edisto-Santee') and f != -1:
        return "South Appalachians"
    elif str(e) in ('SE Plains') and str(h) in ('Alabama','Choctawhatchee-Escambia'):
        return "Mid East Gulf Coastal Plain" 
    elif str(e) in ('SE Plains','Bluff Hills'):
        return "Central Gulf Coastal Plain"
    elif str(h) in ('Mobile-Tombigbee','Pascagoula') and f == -1 :
        return "Central Gulf Coastal Plain"
    elif str(h) in ('Lower Mississippi','Pearl') and f == -1:
        return "Mississippi Alluvial Valley"
    elif str(h) in ('Nueces-Southwestern Texas Coastal','Lower Colorado-San Bernard Coastal','Galveston Bay-San Jacinto','Central Texas Coastal','Louisiana Coastal','Trinity') and f == -1:
        return "Gulf Coastal Prairies"
    elif str(e) in ('High Plains and Tablelands') and str(h) in ('Lower Pecos','Lower Colorado-San Bernard Coastal','Lower Brazos','Middle Brazos','Upper Colorado','Trinity','Brazos Headwaters') and f != -1:
        return "South High Plains and Tablelands"
    elif str(e) in ('High Plains and Tablelands') and f != -1:
        return "North High Plains and Tablelands"
    #elif str(e) in ('Plains and Timbers') and str(h) in ('Lower Pecos','Lower Colorado-San Bernard Coastal','Lower Brazos','Middle Brazos','Upper Colorado','Trinity','Brazos Headwaters') and f != -1:
    #    return "South Plains and Timbers"
    #elif str(e) in ('Plains and Timbers') and f != -1:
    #    return "North Plains and Timbers"
    elif str(h) in ('Lower Rio Grande') and f == -1:
        return "Delete"
    else:
        return str(e) 
"""

In [63]:
# This applies the code above to create the bulk of the eco regions. Because I'm using reclass and if/then, the order
# matters a lot. If we re-order the if/then statements, we can get different results. This is basically saying things like
# if the ecoregion is xx AND the HUC4 is xx, then make it this subregion. I also had to use the FID_eco to deal with
# coastal areas where the ecoregion layer did not cover. Sometimes it was picking those when I didn't want it to.
arcpy.management.CalculateField("ecoH4a", "ecoH4Combine", "Reclass(!EcoCombine!,!NAME!,!FID_eco!)", "PYTHON3", codeblock, "TEXT")     

<Result 'ecoH4a'>

In [64]:
# set codeblock for next step
codeblock = """
def Reclass(e,i,f):
    if str(i) in ('Blue Ridge in N Piedmont') and (f != -1):
        return "North Piedmont"
    elif str(i) in ('Blue Ridge in S Piedmont') and (f != -1):
        return "South Piedmont"
    elif str(i) in ('Edwards Plateau in High Plains and Tablelands') and (f != -1):
        return "South High Plains and Tablelands"
    elif str(i) in ('River Hills in MAV') and (f != -1):
        return "Mississippi Alluvial Valley" 
    elif str(i) in ('MAV in Texas-Louisiana Coast') and (f != -1):
        return "Gulf Coastal Prairies"
    elif str(i) in ('S App in Interior Plateau') and (f != -1):
        return "Interior Plateau"
    elif str(i) in ('WGCP in MAV') and (f != -1):
        return "Mississippi Alluvial Valley"
    elif str(i) in ('N Piedmont in S App') and (f != -1):
        return "South Appalachians"
    elif str(i) in ('MAV in Interior Plateau') and (f != -1):
        return "Interior Plateau"
    elif str(i) in ('N Piedmont in C Atlantic Coastal Plain') and (f != -1):
        return "Central Atlantic Coastal Plain"
    else:
        return str(e)
"""

In [65]:
# this is takign care of the "islands" identifed above
arcpy.management.CalculateField("ecoH4a", "ecoH4Combine", "Reclass(!ecoH4Combine!,!Islands!,!FID_eco!)", "PYTHON3", codeblock, "TEXT")  

<Result 'ecoH4a'>

In [66]:
# set codeblock for next step
codeblock = """
def Reclass(e,i,h,f):
    if (str(i) in ('Blue Ridge in combo N/S Piedmont')) and (str(h) in ('Edisto-Santee','Altamaha-St. Marys','Ogeechee-Savannah') and (f != -1)):
        return "South Piedmont"
    elif (str(i) in ('Blue Ridge in combo N/S Piedmont')) and (str(h) in ('Cape Fear','Pee Dee') and (f == 1813)):
        return "North Piedmont"
    else:
        return str(e)
"""

In [67]:
# this is taking care of the special "island" that crosses 2 final subregions identifed above
arcpy.management.CalculateField("ecoH4a", "ecoH4Combine", "Reclass(!ecoH4Combine!,!Islands!,!Name!,!FID_eco!)", "PYTHON3", codeblock, "TEXT")

<Result 'ecoH4a'>

## Clip out states outside of SECAS
I still have some NE states that are outside of SECAS, I need to clip them out. This is leaving coastal slivers
that I will need to address later

In [68]:
# select states that need to be clipped out
arcpy.analysis.Select(State, "state", "STATEFP in ('10','34','36','42','24')")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\1_SubregionsAndExtent\\SESubregions2022.gdb\\state'>

In [69]:
# erase non SECAS NE states
arcpy.analysis.Erase("ecoH4a", "state", "ecoH4b", None)

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\1_SubregionsAndExtent\\SESubregions2022.gdb\\ecoH4b'>

In [70]:
# select all features that don't have "delete" attribute (this removes some Mexico areas)
arcpy.analysis.Select("EcoH4b", "EcoH4c", "ecoH4Combine <> 'Delete'")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\1_SubregionsAndExtent\\SESubregions2022.gdb\\EcoH4c'>

In [71]:
# dissolve to remove extra lines so I can test simplifying
arcpy.management.Dissolve("EcoH4c", "EcoH4e", "ecoH4Combine", None, "MULTI_PART", "DISSOLVE_LINES")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\1_SubregionsAndExtent\\SESubregions2022.gdb\\EcoH4e'>

## do some testing for slivers and holes

In [72]:
# Multipart to Singlepart
arcpy.management.MultipartToSinglepart("EcoH4e", "EcoH4f")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\1_SubregionsAndExtent\\SESubregions2022.gdb\\EcoH4f'>

In [73]:
# simplify to get rid of interior slivers
arcpy.cartography.SimplifyPolygon("EcoH4f", "sub", "BEND_SIMPLIFY", "1 Meters", "130000000 SquareMeters", "RESOLVE_ERRORS", "KEEP_COLLAPSED_POINTS", None)

id,value
0,E:\WORKING\BaseBlueprint\2022\1_SubregionsAndExtent\SESubregions2022.gdb\sub
1,E:\WORKING\BaseBlueprint\2022\1_SubregionsAndExtent\SESubregions2022.gdb\sub_Pnt


In [74]:
# union with a big square to try to find holes
arcpy.analysis.Union([Square, "sub"], "GapCheck", "ALL", None, "NO_GAPS")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\1_SubregionsAndExtent\\SESubregions2022.gdb\\GapCheck'>

In [75]:
#Look at output by hand at this time to try to find gaps

In [76]:
#didn't find any gaps

In [77]:
# add a field to hold the secondary subregion values
arcpy.management.CalculateField("sub", "subregion2", "!ecoH4Combine!", "PYTHON3", '', "TEXT")

<Result 'sub'>

In [78]:
# dissovle to lump together Puerto Rico and VI
arcpy.management.Dissolve("sub", "sub2", "subregion2", None, "MULTI_PART", "DISSOLVE_LINES")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\1_SubregionsAndExtent\\SESubregions2022.gdb\\sub2'>

## Add marine ecoregions for South Atlantic

In [79]:
# pull out relevant level 1 marine subrgions
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SA):
    arcpy.analysis.Select(MaR, "MarineSub", "NAME_L1 IN ('Carolinian Atlantic', 'Gulf Stream', 'Northern Gulf Stream Transition', 'Virginian Atlantic')")

In [80]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 'Carolinian Atlantic' or value == 'Virginian Atlantic':
        return 'Marine Shelf and Extension'
    elif value == 'Gulf Stream' or value == 'Northern Gulf Stream Transition':
        return 'Marine Gulf Stream'    
"""

In [81]:
# add and calculate a field to be used to dissolve
arcpy.management.CalculateField("MarineSub", "dissolve", "Reclass(!NAME_L1!)", "PYTHON3", codeblock, "TEXT")

<Result 'MarineSub'>

In [82]:
# perform dissolve
arcpy.management.Dissolve("MarineSub", "MarineSub_Dissolve", "dissolve", None, "MULTI_PART", "DISSOLVE_LINES")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\1_SubregionsAndExtent\\SESubregions2022.gdb\\MarineSub_Dissolve'>

In [83]:
# ugh, so many slivers! Change strategy. I'm going to use a buffer to try to get rid of slivers
arcpy.analysis.Select("MarineSub_Dissolve", "GulfStream", "dissolve = 'Marine Gulf Stream'")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\1_SubregionsAndExtent\\SESubregions2022.gdb\\GulfStream'>

In [84]:
# add a buffer to try to remove slivers
arcpy.analysis.Buffer("GulfStream", "GulfStream_Buffer100", "100 Meters", "FULL", "ROUND", "ALL", None, "PLANAR")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\1_SubregionsAndExtent\\SESubregions2022.gdb\\GulfStream_Buffer100'>

In [85]:
# do negative buffer to try to get back to rough original
arcpy.analysis.Buffer("GulfStream_Buffer100", "GulfStream_Buffer100neg100", "-100 Meters", "FULL", "ROUND", "ALL", None, "PLANAR")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\1_SubregionsAndExtent\\SESubregions2022.gdb\\GulfStream_Buffer100neg100'>

In [86]:
# add field
# add a field to hold the secondary subregion values
arcpy.management.CalculateField("GulfStream_Buffer100neg100", "subregion2", "'Marine Gulf Stream'", "PYTHON3", '', "TEXT")

<Result 'GulfStream_Buffer100neg100'>

In [87]:
# Select shelf so I can do the same buffer on shelf, only this time I'll do 200  meters to close the gap between
# the shelf and gulf stream. I also won't do a negative buffer, because I don't need to snap these lines back, they
# will be removed where they overlap the inland subregions
arcpy.analysis.Select("MarineSub_Dissolve", "Shelf", "dissolve = 'Marine Shelf and Extension'")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\1_SubregionsAndExtent\\SESubregions2022.gdb\\Shelf'>

In [88]:
# add a buffer to try to remove slivers
arcpy.analysis.Buffer("Shelf", "Shelf_Buffer200", "200 Meters", "FULL", "ROUND", "ALL", None, "PLANAR")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\1_SubregionsAndExtent\\SESubregions2022.gdb\\Shelf_Buffer200'>

In [89]:
# add field
# add a field to hold the secondary subregion values
arcpy.management.CalculateField("Shelf_Buffer200", "subregion2", "'Marine Shelf and Extension'", "PYTHON3", '', "TEXT")

<Result 'Shelf_Buffer200'>

In [90]:
# use update to combine the 2 layers and keep one attribute for subregion2
arcpy.analysis.Update("GulfStream_Buffer100neg100", "Shelf_Buffer200", "Update1", "BORDERS", None)

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\1_SubregionsAndExtent\\SESubregions2022.gdb\\Update1'>

In [91]:
# union with a big square to try to find holes
arcpy.analysis.Union([Square, "Update1"], "GapCheck", "ALL", None, "NO_GAPS")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\1_SubregionsAndExtent\\SESubregions2022.gdb\\GapCheck'>

In [92]:
# clip to extent of South Atlantic
arcpy.analysis.Clip("Update1", SA, "SAmarine", None)

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\1_SubregionsAndExtent\\SESubregions2022.gdb\\SAmarine'>

In [105]:
# Multipart to Singlepart just to make sure no slivers
arcpy.management.MultipartToSinglepart("SAmarine", "SAmarine2")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\1_SubregionsAndExtent\\SESubregions2022.gdb\\SAmarine2'>

# combine inland and marine subregions

In [106]:
# use update to combine the 2 layers and keep one attribute for subregion2
arcpy.analysis.Update("SAmarine2", "sub2", "SEBlueprintSubRgn1", "BORDERS", None)

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\1_SubregionsAndExtent\\SESubregions2022.gdb\\SEBlueprintSubRgn1'>

In [107]:
# set codeblock for next step
codeblock = """
def Reclass(s,s2):
    if str(s2) in ('North Atlantic Coastal Plain','Central Atlantic Coastal Plain','South Atlantic Coastal Plain'):
        return "Atlantic Coastal Plain"
    #elif str(s2) in ('North Appalachians','South Appalachians'):
    #    return "Appalachians"
    elif str(s2) in ('North Piedmont','South Piedmont'):
        return "Piedmont"
    elif str(s2) in ('South High Plains and Tablelands','North High Plains and Tablelands'):
        return "High Plains and Tablelands"
    #elif str(s2) in ('South Plains and Timbers','North Plains and Timbers'):
    #    return "Plains and Timbers"
    #elif str(s2) in ('Central Gulf Coastal Plain','Mid East Gulf Coastal Plain','East Gulf Coastal Plain Sub'):
    #    return "East Gulf Coastal Plain"
    else:
        return str(s2)
"""

In [108]:
# Add another field to hold the larger, combined subregion names
arcpy.management.CalculateField("SEBlueprintSubRgn1", "subregion", "Reclass(!subregion!,!subregion2!)", "PYTHON3", codeblock, "TEXT")  

<Result 'SEBlueprintSubRgn1'>

In [109]:
# Add final fields, so they will be in order
arcpy.management.CalculateField("SEBlueprintSubRgn1", "SubRgn", "!subregion!", "PYTHON3", '', "TEXT")
arcpy.management.CalculateField("SEBlueprintSubRgn1", "SubRgn_II", "!subregion2!", "PYTHON3", '', "TEXT")
arcpy.management.CalculateField("SEBlueprintSubRgn1", "SECAS", "'SECAS'", "PYTHON3", '', "TEXT")

<Result 'SEBlueprintSubRgn1'>

In [110]:
# delete extra fields
arcpy.management.DeleteField("SEBlueprintSubRgn1", "subregion2;ORIG_FID;subregion")

<Result 'SEBlueprintSubRgn1'>

In [119]:
# Dissolve to save the subregions we actually used in zonation
arcpy.management.Dissolve("SEBlueprintSubRgn1", "BaseBlueprintSubRgnPlusCar", "SubRgn", None, "MULTI_PART", "DISSOLVE_LINES")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\1_SubregionsAndExtent\\SESubregions2022.gdb\\BaseBlueprintSubRgnPlusCar'>

In [120]:
# selecte the draft base blueprint area (not PR and VI)
arcpy.analysis.Select("SEBlueprintSubRgn", "BaseBlueprintSubRgn", "SubRgn <> 'Puerto Rico and Virgin Islands'")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\1_SubregionsAndExtent\\SESubregions2022.gdb\\BaseBlueprintSubRgn'>

## Export

In [121]:
#Export to shapefile
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.conversion.FeatureClassToShapefile("BaseBlueprintSubRgn", ShapeOut)

In [123]:
#Export SE blueprint subregions to shapefile
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.conversion.FeatureClassToShapefile("BaseBlueprintSubRgnPlusCar", ShapeOut)

In [ ]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=snap, snapRaster=SA, cellSize=SA):
    arcpy.conversion.PolygonToRaster("BaseBlueprintSubRgn", "SECAS", ExtentOut, "CELL_CENTER", "NONE", 30)